In [1]:
import pandas as pd
import re
import networkx as nx

>* We are going back to week 6 when we analyzed email data to build edges between users. We will use the data from week 6, which we used the feather format to store the data. We will use the same data to build a graph and analyze it.

In [3]:
data=pd.read_feather('../week6/sample-feather.feather')

>* Let's check the columns

In [7]:
data.columns

Index(['thread_id', 'thread_name', 'body', 'account', 'url', 'date',
       'longevity'],
      dtype='object')

>* In `data` dataframe, we have six columns, each representing as follows:
    
    * `thread_id` : unique id for each thread
    * `thread_name` : the first subject of the email
    * `body` : the content of the email 
    * `account` : the email account of the sender 
    * `url` : the url of the email
    * `date` : the date of the email 

In [9]:
data.head(5)

,thread_id,thread_name,body,account,url,date,longevity
0,5608,Tool-to-log-all-messages-targeting-a-given-window,"[Hello all, Does someone here know of a tool t...","[webmaster, jtwauthier, soronel.haetir]",[https://www.freelists.org/post/program-l/Tool...,"[2016-01-19T10:12:01.000000000, 2016-01-19T15:...",0 days 06:12:00
1,6826,html5js-newby,"[HI All, I'm trying to create a demo for aira ...","[blindwiz, taksantong, blindwiz]",[https://www.freelists.org/post/program-l/html...,"[2019-02-20T07:34:07.000000000, 2019-02-20T17:...",0 days 13:54:20
2,4209,Searching-in-dataTables-vbnet,[Hi all. A dataTable is the registers of one t...,"[pmorales, ofbgmail, pmorales, ofbgmail, pmora...",[https://www.freelists.org/post/program-l/Sear...,"[2013-09-02T23:01:35.000000000, 2013-09-03T15:...",5 days 17:13:11
3,2119,Ot-looking-for-an-app,"[Hello, I've got an assignment for management ...","[compgeek13, justind, wunderg]",[https://www.freelists.org/post/program-l/Ot-l...,"[2008-11-11T00:52:32.000000000, 2008-11-11T13:...",0 days 14:42:00
4,2906,Learn-Python-The-Hard-Way-Second-Edition,"[Hi, For those interested, Learn Python Hard W...","[james.homme, jr]",[https://www.freelists.org/post/program-l/Lear...,"[2011-12-05T12:45:48.000000000, 2011-12-05T16:...",0 days 04:05:30


> * Let's build edges between the users in the `account` column.

In [10]:
#We will need a combination of all the accounts in the 'account' column to create the edges of the graph
#We will use itertools.combinations to create the combination

import itertools
edges=[]
for idx, val in data['account'].items():
    edges.extend(list(itertools.combinations(val, 2)))

In [11]:
edges[:5]

[('webmaster', 'jtwauthier'),
 ('webmaster', 'soronel.haetir'),
 ('jtwauthier', 'soronel.haetir'),
 ('blindwiz', 'taksantong'),
 ('blindwiz', 'blindwiz')]

>* Let's get rid of the self-loops.

In [12]:
edges_loop = [edge for edge in edges if edge[0] != edge[1]]

In [13]:
edges_loop[:5]

[('webmaster', 'jtwauthier'),
 ('webmaster', 'soronel.haetir'),
 ('jtwauthier', 'soronel.haetir'),
 ('blindwiz', 'taksantong'),
 ('taksantong', 'blindwiz')]

>* We will generate an empty graph object `G` and populate the graph with the edges.

In [23]:
G=nx.Graph()

In [24]:
G.add_edges_from(edges_loop)

>* When you add edges, the graph object will add the nodes automatically.

In [25]:
len(list(G.nodes))

140

>* Thus far, we calculated the degree of the nodes by hardcoding the values. But we can use the built-in function of networkx to calculate various centrality measures.

>* Degree centrality: The number of edges that are connected to the node.
>* Betweenness centrality: The number of times the node acts as a bridge along the shortest path between two other nodes.
>* Closeness centrality: The average length of the shortest path between the node and all other nodes.

>* Degree centrality: Node connectivity, local influence
>* Betweenness centrality: Bridging roles, broker
>* Closeness centrality: Proximity to other nodes, efficient communication

In [28]:
nx.degree_centrality(G)['webmaster'] #degree centrality

0.17266187050359713

In [29]:
nx.betweenness_centrality(G)['webmaster'] #betweenness centrality
#e-06 means 10^-6 or 0.000001

0.028790642170402562

In [30]:
nx.closeness_centrality(G)['webmaster'] #closeness centrality

0.4527687296416938

>* Q. Who are the top 5 usernames who have the highest degree centrality?

In [31]:
sorted(nx.degree_centrality(G).items(), key=lambda x:x[1], reverse=True)[:5]

[('jacobk', 0.30935251798561153),
 ('compgeek13', 0.2949640287769784),
 ('travis', 0.2517985611510791),
 ('florianbeijers', 0.23741007194244607),
 ('soronel.haetir', 0.22302158273381295)]

>* Let's put the degree centrality score as a node attribute called `degree`.
>* To do so, we use the `nx.set_node_attributes` method.

In [32]:
nx.set_node_attributes(G, nx.degree_centrality(G), 'degree')

>* Q. Who are the top 5 usernames who have the highest betweenness centrality?

In [33]:
sorted(nx.betweenness_centrality(G).items(), key=lambda x:x[1], reverse=True)[:5]

[('jacobk', 0.16864972604002265),
 ('compgeek13', 0.16515285378497213),
 ('soronel.haetir', 0.13470213623000243),
 ('travis', 0.13201814940116208),
 ('dzhovani.chemishanov', 0.12660085241275623)]

>* Let's put the betweenness centrality score as a node attribute called `betweenness`.
>* To do so, we use the `nx.set_node_attributes` method.

In [34]:
nx.set_node_attributes(G, nx.betweenness_centrality(G), 'betweenness')

>* Q. Who are the top 5 usernames who have the highest closeness centrality?

In [35]:
sorted(nx.closeness_centrality(G).items(), key=lambda x:x[1], reverse=True)[:5]

[('compgeek13', 0.5387596899224806),
 ('jacobk', 0.5305343511450382),
 ('travis', 0.5148148148148148),
 ('dzhovani.chemishanov', 0.486013986013986),
 ('soronel.haetir', 0.4843205574912892)]

>* Let's put the closeness centrality score as a node attribute called `closeness`.
>* To do so, we use the `nx.set_node_attributes` method.

In [36]:
nx.set_node_attributes(G, nx.closeness_centrality(G), 'closeness')

In [37]:
G.nodes['travis']

{'degree': 0.2517985611510791,
 'betweenness': 0.13201814940116208,
 'closeness': 0.5148148148148148}

>* Let's have another node attribute called `role`.
>* If the user initiated the email conversation thread, we will assign the value `asker`.
>* If the user replied to the email conversation thread, we will assign the value `answerer`.
>* No matter `asker` replied to the email conversation thread later, we will still assign the value `asker`.

In [50]:
asker=data['account'].apply(lambda x: x[0]) # find the first element in the column 'account'.

In [53]:
len(set(asker)) #there are 59 unique accounts whose role is 'asker'

59

In [57]:
role={}
for val in [i for sublist in data['account'] for i in sublist]:
    if val in set(asker):
        role[val]='asker'
    else:
        role[val]='answerer'

In [58]:
nx.set_node_attributes(G, role, 'role')

In [59]:
G.nodes['travis']

{'degree': 0.2517985611510791,
 'betweenness': 0.13201814940116208,
 'closeness': 0.5148148148148148,
 'ei_idx': 1.0,
 'role': 'answerer'}

>* Homophily is the tendency of individuals to associate and bond with similar others.
>* We can measure homophily by comparing the number of edges between nodes of the same type to the number of edges between nodes of different types.
>* One of popular ways to measure the node-level homophily is to calculate the E-I index proposed by Krackhardt and Stern (1988).
>* https://doi.org/10.2307/2786835

<img src="../week8/ei-index.png" width=500px height=500px />

>* Unfortunately, networkx does not have a built-in function to calculate the E-I index.
>* We will make a function to calculate the E-I index.

In [62]:
def ego_EI_idx(graph_object):
    EI_dic = {}
    for k in graph_object.nodes:
        external = 0
        internal = 0
        try:
            for i in graph_object.edges(k):
                if graph_object.nodes[i[1]]['role'] == graph_object.nodes[k]['role']:
                    internal += 1
                else:
                    external += 1
        except ZeroDivisionError:
            pass
        if external + internal != 0:
            EI_dic[k] = (external - internal) / (external + internal)
        else:
            EI_dic[k] = 0
    nx.set_node_attributes(graph_object, EI_dic, name="ei_idx")

In [63]:
ego_EI_idx(G)

In [64]:
G.nodes['webmaster']

{'degree': 0.17266187050359713,
 'betweenness': 0.028790642170402562,
 'closeness': 0.4527687296416938,
 'ei_idx': 0.25,
 'role': 'asker'}

>* In order to calculate the graph-level homophily, there is another method called `assortativity coefficient`.
>* The assortativity coefficient is a measure used to quantify the degree to which nodes in a network tend to be connected to other nodes that are similar or dissimilar.
> * 1: Perfect assortative
> * -1: Perfect disassortative

In [65]:
nx.attribute_assortativity_coefficient(G, 'role')

-0.0865055528392819

>* If we want to measure the level of clustering in the network, we can use (1) transitivity and (2) clustering coefficient.
>* `Transitivity` is the ratio of triangles to triplets in the network.
>* `Clustering coefficient` is the clustering coefficient of the node.
>* `Average clustering` is the average clustering coefficient of all the nodes in the network (Graph-level clustering).

> * Why do we look at `triangles` in the network?
>* https://faculty.ucr.edu/~hanneman/nettext/C8_Embedding.html
>* https://bryangraham.github.io/econometrics/downloads/working_papers/DynamicNetworks/Homophily_and_Transitivity_April2016.pdf

<img src="../week8/transitivity.png" width=3500px height=80px />

<img src="../week8/transitivity-figure.png" width=800px height=300px />

<img src="../week8/transitivity-1.png" width=700px height=70px />

<img src="../week8/transitivity-2.png" width=700px height=120px />

<img src="../week8/transitivity-3.png" width=700px height=120px />

<img src="../week8/triadic_closure.png" width=300px height=700px />

In [66]:
nx.transitivity(G) #transitivity 

0.4461376773515502

<img src="../week8/transitivity-metric.png" width=300px height=100px />

In [ ]:
nx.clustering(G_eiindex) #clustering coefficient

<img src="../week8/clustering-metric.png" width=500px height=100px />

In [ ]:
nx.average_clustering(G_eiindex) #average clustering coefficient

>* Louvain community detection algorithm is a method to detect the communities in the network.
>* First, it iteratively optimizes the modularity score of the network by moving nodes between communities.
>* Modularity:a measure of the structure of networks or graphs which measures the strength of division of a network into modules (also called groups, clusters or communities).
>* Second, it stops when the modularity score cannot be increased further.
>* Third, it returns the communities as the output.

<img src="../week8/louvain_community.png" width=700px height=300px />

>* `pip install community`
>* `pip install python-louvain`

In [ ]:
import community.community_louvain
len(nx.community.louvain_communities(G)) #community detection

#### Practice

In [ ]:
data=pd.read_csv('../week5/subset-2021-01-11-voter_fraud.csv')

>* Let's see what it has for column names.

In [ ]:
#YOUR CODE HERE

>* Let's subset the data to have only `bodywithurls`, `username`, `followers`, and `following` columns.

In [ ]:
#YOUR CODE HERE

>* Q. Print the first 5 rows of the DataFrame.

In [ ]:
#YOUR CODE HERE

>* Make the `username` column lowercase.

In [ ]:
#YOUR CODE HERE

> * Looks like there are duplicates in the dataset. Let's remove the duplicates.

In [ ]:
#YOUR CODE HERE

> * Let's extract the mentions from the `bodywithurls` column and create a new column `mentions` with the mentions.

In [ ]:
#YOUR CODE HERE

> * Let's build edges between the users who have mentioned.
> * To do so, we will use the `mentions` column and iterate over the rows to create edges between the users in the `mention` column.

In [ ]:
#YOUR CODE HERE

>* Create an empty graph object `P` 

In [ ]:
#YOUR CODE HERE

>* Populate the graph `p` with the edges (where you put the tuples to represent the edges)
>* You may have to use the `add_edges_from` method to add the edges to the graph.

In [ ]:
#YOUR CODE HERE

>* How many unique nodes are there in the graph `P`?
>* Use `.nodes()` method to get the unique nodes.

In [ ]:
#YOUR CODE HERE

>* In the DataFrame, there is a column called `followers` and `following`.
>* Let's add the `followers` and `following` as the node attributes to the graph `P`.
>* Remember how to deal with the nodes that are not in `username` column. If the node is not in the `username` column, add 0 for `followers` and `following` attributes.

In [ ]:
#YOUR CODE HERE

>* Calculate the degree centrality of the graph `P` and assign the value of the degree centrality to the node as the node attribute.
>* Use the `nx.degree_centrality` method to calculate the degree centrality.
>* The name of attribute should be `degree_centrality`.

In [ ]:
#YOUR CODE HERE